<a href="https://colab.research.google.com/github/Feekah/HumanPoseEstimation/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc

#Define Functions

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

def init_csv(dataset_path: str):
    '''
    Create a blank csv file with just columns
    '''

    # Write all the columns to a file
    with open(dataset_path, mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

#define a function to calculate angles between joints
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle >180.0:
        angle = 360-angle

    return angle

def export_landmark_to_csv(dataset_path: str, results, position: str, angle, angle2, angle3, label: str) -> None:
    '''
    Export Labeled Data from detected landmark to csv
    '''
    landmarks = results.pose_landmarks.landmark
    keypoints = []

    try:
        # Extract coordinate of important landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
            keypoints.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        keypoints = list(np.array(keypoints).flatten())

        # Insert action as the label (first column)
        keypoints.insert(0, position)
        keypoints.insert(1, angle)
        keypoints.insert(2, angle2)
        keypoints.insert(3, angle3)
        keypoints.insert(4, label)

        # Append new row to .csv file
        with open(dataset_path, mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)


    except Exception as e:
        print(e)
        pass

def describe_dataset(dataset):
    ''''''

    data = pd.read_csv(dataset)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Position: \n{data['Label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {duplicate.sum(axis=1)}")

    return data

def process_single_video(video, csv_path, video_label):
    cap = cv2.VideoCapture(video)
    init_csv(csv_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            # Reduce size of a frame
            image = rescale_frame(image, 50)
            image = cv2.flip(image, 1)

            # Recolor image from BGR to RGB for mediapipe
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            results = pose.process(image)

            # Recolor image from BGR to RGB for mediapipe
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            lms = results.pose_landmarks.landmark

            hip = [lms[mp_pose.PoseLandmark.LEFT_HIP.value].x,lms[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [lms[mp_pose.PoseLandmark.LEFT_KNEE.value].x,lms[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            shoulder = [lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            foot_index = [lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            angle2 = calculate_angle(shoulder, hip, knee)
            angle3 = calculate_angle(knee, ankle, foot_index)

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            cv2.imshow("CV2", image)

            # Pressed key for action
            k = cv2.waitKey(1) & 0xFF

            if angle < 150:
                export_landmark_to_csv(csv_path, results, "down", angle, angle2, angle3, video_label)
            elif angle > 150:
                export_landmark_to_csv(csv_path, results, "up", angle, angle2, angle3, video_label)
            # Press s to stop
            elif k == ord("s"):
                break
            else: continue

        cap.release()
        cv2.destroyAllWindows()

        # (Optional)Fix bugs cannot close windows in MacOS (https://stackoverflow.com/questions/6116564/destroywindow-does-not-close-window-on-mac-using-python-and-opencv)
        for i in range (1, 5):
            cv2.waitKey(1)

def evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms):
    # Initialize dictionaries to store metric results
    accuracy_results = {}
    precision_results = {}
    recall_results = {}
    f1_results = {}

    # Iterate through algorithms and evaluate performance
    for name, model in algorithms:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy_results[name] = accuracy_score(y_test, y_pred)
        precision_results[name] = precision_score(y_test, y_pred, average='weighted')
        recall_results[name] = recall_score(y_test, y_pred, average='weighted')
        f1_results[name] = f1_score(y_test, y_pred, average='weighted')

    # Print the results
    print("Algorithm\tAccuracy\tPrecision\tRecall\t\tF1-score")
    for name in algorithms:
        print(f"{name[0]}\t\t{accuracy_results[name[0]]:.4f}\t\t{precision_results[name[0]]:.4f}\t\t{recall_results[name[0]]:.4f}\t\t{f1_results[name[0]]:.4f}")



In [3]:
#Intialize the Mediapipe libraries
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Highlight the important joints for the squat exercise

# Determine important landmarks for squat
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "LEFT_HIP",
    "LEFT_KNEE",
    "LEFT_ANKLE",
    "LEFT_FOOT_INDEX"
]

# Generate all columns of the data frame

landmarks = ["Position", "Hip_Knee-Ankle_angle", "Shoulder_hip_knee_angle", "Knee_ankle_foot_angle", "Label"] # Label column

for lm in IMPORTANT_LMS:
    landmarks += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

video_labels = {
    "Correct_Squat_Female.mp4": "Correct Pose",
    "Correct_Squat_Female_2.mp4": "Correct Pose",
    "Correct_Squat_Male.mp4" : "Correct Pose",
    "Incorrect_Back_Female.mp4" : "Low back",
    "Incorrect_Back_Male.mp4" : "Low back",
    "Incorrect_Dorsiflexion_Female.mp4" : "Knee too forward",
    "Incorrect_Dorsiflexion_Male.mp4" : "Knee too forward",
    "Incorrect_Halfway_Female.mp4" : "Early stopping",
    "Incorrect_Halfway_Male.mp4": "Early stopping"

    # ... add other videos and their corresponding labels ...
}

videos = [
    "Correct_Squat_Female.mp4",
    "Correct_Squat_Female_2.mp4",
    "Correct_Squat_Male.mp4",
    "Incorrect_Back_Female.mp4",
    "Incorrect_Back_Male.mp4",
    "Incorrect_Dorsiflexion_Female.mp4",
    "Incorrect_Dorsiflexion_Male.mp4",
    "Incorrect_Halfway_Female.mp4",
    "Incorrect_Halfway_Male.mp4"
    # ... add paths to your other videos here ...
]

output_csv = "Documents/PEVideos/csvfiles/traindata.csv"

for video in videos:
    csv_filename = f"{video.split('.')[0]}.csv"
    label = video_labels.get(video, "Unknown Pose")
    process_single_video(video, csv_filename, label)

# Get a list of all generated CSV files
csv_files = [f"{video.split('.')[0]}.csv" for video in videos]

# Concatenate CSV files with the same headers
concatenated_data = pd.concat([pd.read_csv(csv_file) for csv_file in csv_files], ignore_index=True)
concatenated_data.to_csv(output_csv, index=False)

describe_dataset(output_csv)




INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Headers: ['Position', 'Hip_Knee-Ankle_angle', 'Shoulder_hip_knee_angle', 'Knee_ankle_foot_angle', 'Label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'left_foot_index_x', 'left_foot_index_y', 'left_foot_index_z', 'left_foot_index_v']
Number of rows: 4786 
Number of columns: 29

Position: 
Correct Pose        1760
Low back            1123
Knee too forward     968
Early stopping       935
Name: Label, dtype: int64

Missing values: False

Duplicate Rows : Series([], dtype: float64)


,Position,Hip_Knee-Ankle_angle,Shoulder_hip_knee_angle,Knee_ankle_foot_angle,Label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,...,left_knee_z,left_knee_v,left_ankle_x,left_ankle_y,left_ankle_z,left_ankle_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v
0,up,175.059343,173.198858,91.206212,Correct Pose,0.570615,0.151710,-0.013997,0.999820,0.469433,...,0.432409,0.281333,0.464446,0.751567,0.694830,0.543678,0.577964,0.756860,0.645782,0.746166
1,up,174.528748,173.151824,90.678607,Correct Pose,0.570581,0.151545,-0.013209,0.999823,0.470826,...,0.384174,0.289797,0.464899,0.756392,0.607728,0.549365,0.580712,0.761637,0.536170,0.749270
2,up,174.242538,173.648651,90.642115,Correct Pose,0.569845,0.150000,-0.025548,0.999824,0.472668,...,0.360974,0.296625,0.465901,0.758756,0.574311,0.553589,0.583289,0.764969,0.497071,0.751713
3,up,174.370524,174.846017,90.388832,Correct Pose,0.566328,0.149965,-0.024779,0.999823,0.473273,...,0.328178,0.301836,0.466321,0.763183,0.518508,0.555534,0.586006,0.769756,0.436164,0.752867
4,up,174.405288,175.401627,90.830804,Correct Pose,0.562791,0.149473,-0.030604,0.999818,0.474346,...,0.315472,0.306432,0.466623,0.767509,0.503405,0.556280,0.586629,0.775575,0.414472,0.752391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781,up,169.143312,170.045915,90.468519,Early stopping,0.541041,0.168794,-0.121075,0.998308,0.471827,...,0.398843,0.351821,0.500592,0.714640,0.655318,0.622275,0.603222,0.730298,0.560591,0.782365
4782,up,169.158608,169.838029,90.516055,Early stopping,0.541413,0.168808,-0.120032,0.998254,0.471909,...,0.401171,0.347995,0.500456,0.715304,0.663671,0.614444,0.603424,0.731306,0.566797,0.775577
4783,up,169.755366,169.665195,91.577726,Early stopping,0.541658,0.168955,-0.147604,0.998278,0.475039,...,0.401003,0.342334,0.500420,0.716364,0.665557,0.602972,0.603531,0.734436,0.571407,0.766013
4784,up,169.139260,169.621292,91.437551,Early stopping,0.542056,0.168969,-0.186322,0.998371,0.475399,...,0.378132,0.338771,0.500831,0.716648,0.628127,0.594054,0.604929,0.735516,0.550539,0.756928


#Model Training

In [5]:
# Define algorithms
algorithms = [
    #("LR", LogisticRegression(max_iter=1000)),
    ("SVC", SVC(probability=True)),
    ('KNN', KNeighborsClassifier()),
    ("DTC", DecisionTreeClassifier()),
    ("SGDC", CalibratedClassifierCV(SGDClassifier())),
    ("NB", GaussianNB()),
    ('RF', RandomForestClassifier()),
]

In [4]:
data = pd.read_csv("concat2.csv")

#Using Label - Encoding

In [6]:
le = data.copy()

# Separate categorical columns
categorical_columns = ['Position', 'Label']  # Add more columns if needed

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
for col in categorical_columns:
    le[col] = label_encoder.fit_transform(le[col])

# Extract features and class
X = le.drop("Label", axis=1) # features
y = le["Label"]

# Split train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
y_test.head(3)


4062    1
3064    2
292     0
Name: Label, dtype: int64

In [7]:
# Call the function
evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms)

Algorithm	Accuracy	Precision	Recall		F1-score
SVC		0.5616		0.7131		0.5616		0.5357
KNN		0.7938		0.7977		0.7938		0.7937
DTC		0.9866		0.9866		0.9866		0.9866
SGDC		0.5134		0.5868		0.5134		0.4426
NB		0.8096		0.8172		0.8096		0.8110
RF		1.0000		1.0000		1.0000		1.0000


#Using One-Hot Encoding

In [162]:
oh = data.copy()
# Separate categorical columns
categorical_columns = ['Position']  # Add more columns if needed

# Apply one-hot encoding to categorical columns
df_encoded = pd.get_dummies(oh, columns=categorical_columns)

df_encoded['Label'] = label_encoder.fit_transform(df_encoded['Label'])

df_encoded.head()

# Extract features and class
X = le.drop("Label", axis=1) # features
y = le["Label"]

# Split train set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

4062    1
3064    2
292     0
Name: Label, dtype: int64

In [163]:
# Call the function
evaluate_algorithms(X_train, y_train, X_test, y_test, algorithms)

Algorithm	Accuracy	Precision	Recall		F1-score
SVC		0.5770		0.7210		0.5770		0.5521
KNN		0.8092		0.8157		0.8092		0.8097
DTC		0.9858		0.9858		0.9858		0.9858
SGDC		0.5533		0.6915		0.5533		0.5132
NB		0.7938		0.7977		0.7938		0.7946
RF		1.0000		1.0000		1.0000		1.0000


In [8]:
model1 = DecisionTreeClassifier().fit(X, y)
        #y_pred = model.predict(X_test)

In [9]:
model_filename = 'Documents/PEVideos/trained_model.pkl'
joblib.dump(model1, model_filename)

['Documents/PEVideos/trained_model.pkl']

In [ ]:
cap = cv2.VideoCapture(0)

init_csv(csv_doc)

squat_counter = 0
squat_stage = 'Up'

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
      ret, image = cap.read()
      # Recolor image to RGB
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      image.flags.writeable = False

      # Make detection
      results = pose.process(image)

      # Recolor back to BGR
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      # Extract landmarks
      try:


        lms = results.pose_landmarks.landmark

        hip = [lms[mp_pose.PoseLandmark.LEFT_HIP.value].x,lms[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [lms[mp_pose.PoseLandmark.LEFT_KNEE.value].x,lms[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,lms[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        shoulder = [lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,lms[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        foot_index = [lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,lms[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

        # Calculate angle
        angle = calculate_angle(hip, knee, ankle)
        angle2 = calculate_angle(shoulder, hip, knee)
        angle3 = calculate_angle(knee, ankle, foot_index)

        # Draw landmarks and connections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Display the saved count
        cv2.putText(image, f"UP saved: {squat_counter}", (50, 150), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f"DOWN saved: {squat_counter}", (50, 200), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)

        cv2.imshow("CV2", image)

        if angle < 150 and squat_stage == 'Up':
          export_landmark_to_csv(csv_doc, results, "down", angle, angle2, angle3, video_label)
          squat_stage == 'Down'
        elif angle > 70 and squat_stage == 'Down':
          export_landmark_to_csv(csv_doc, results, "up", angle, angle2, angle3, video_label)
          squat_stage == 'Up'
          squat_counter +=1

            # Press q to stop
        elif k == ord("q"):
          break
        else: continue

      except:
            pass
